In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
##Import thư viện, mappingpath
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from matplotlib import pyplot as plt
import random
import cv2
from glob import glob
import random

import tensorflow as tf
import keras
from keras.models import *
from keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adamax
from keras.applications.efficientnet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.applications.efficientnet import EfficientNetB7
from IPython.display import display
from PIL import Image

In [ ]:
mapping_path = '/kaggle/input/imagenet1kmediumtrain-100k/LOC_synset_mapping.txt'
src_path_train = "/kaggle/input/imagenet1kmediumtrain-100k/train_100000/train_100000"
batch_size = 128

In [ ]:
#PathFile

class_mapping_dict = {}
i = 0
for line in open(mapping_path):
    if 0 <= i < 1000:
        label = line[9:].strip()
        class_mapping_dict[i] = label
    i += 1

In [ ]:
image_gen = ImageDataGenerator(
    horizontal_flip=True,
    preprocessing_function = preprocess_input,
    validation_split=0.20)

train_generator = image_gen.flow_from_directory(
  src_path_train,
  target_size=(300,300),
  batch_size=batch_size,
  subset="training",

)

test_generator = image_gen.flow_from_directory(
  src_path_train,
  target_size=(300,300),
  batch_size=batch_size,
  subset="validation",


)

In [ ]:
#Tao model
model_efficientnet = EfficientNetB7(
    weights='imagenet',
    include_top=False,
    input_shape=(300, 300, 3),
    pooling="max"
)

# model_efficientnet.trainable = True
for layer in model_efficientnet.layers:
    layer.trainable = False

x = model_efficientnet.output
x = BatchNormalization()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(512, activation="relu")(x)
x = Dropout(0.3)(x)
x = Dense(128, activation="relu")(x)
x = Dropout(0.3)(x)
outputs = Dense(len(class_mapping_dict), activation="softmax")(x)

model = Model(inputs=model_efficientnet.input, outputs=outputs)
lr = 0.001

model.compile(optimizer=Adamax(learning_rate=lr), loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
## Load lại model

## Đường dẫn đến file model
model_path = "/kaggle/input/model123/efficientnetb7_model.h5"

# # Tải model
model = keras.models.load_model(model_path)

In [ ]:
# Train model

from tensorflow.keras.callbacks import ModelCheckpoint
ckpoint = ModelCheckpoint("/kaggle/working/model_ImagenetB7.h5", monitor="val_accuracy", save_best_only=True, mode="auto")

n_epochs = 12
hist = model.fit(
    train_generator,
    epochs=n_epochs,
    validation_data=test_generator,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator),
    callbacks=[ckpoint]
)

In [ ]:
accuracy = hist.history['accuracy']
val_accuracy = hist.history['val_accuracy']

loss = hist.history['loss']
val_loss = hist.history['val_loss']

epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'b', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'r', label='Validation accuracy')

plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'b', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')

plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
evaluation_dico = model.evaluate(test_generator, batch_size = 128, verbose = 0, return_dict=True)
print('model_ImageNetB7 :')
print('Loss : ' + str(evaluation_dico['loss']))
print('Accuracy : ' + str(evaluation_dico['accuracy']))